In [ ]:
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
from specim.specfuncs import spec1d
import numpy as np
import matplotlib.pyplot as plt
import glob
from random import sample
import pandas as pd
import seaborn as sn
import pm_veldis_util as vd
from collections import Counter
from keckcode.deimos import deimosmask1d

In [ ]:
coadded_spectra_red = deimosmask1d.DeimosMask1d('../galaxy_spectra/0712m4_coadd.fits')

In [ ]:
specid_red = [] #np.zeros(coadded_spectra.nspec)
info = coadded_spectra_red.slitinfo
#print(info)
for i in range(coadded_spectra_red.nspec):
    spec_id = '%d_%d_%d_%d' % (info['det'][i], info['slitid'][i], info['objid'][i], info['spatloc'][i])
    specid_red.append(spec_id)
    print(spec_id)

In [ ]:
gal_spec_id_red = specid_red[1]
print('galaxy_spectra_id: %s' % (gal_spec_id_red))
data_red =  coadded_spectra_red[gal_spec_id_red]
#print(data)
data_red.smooth(7)
data_red.mark_lines('abs', z=0.405, usesmooth=True)
data_red.save('0712_lens_spectra_red_side.txt', outformat='text')
#lamda_galaxy = data['wav']
#spectra_galaxy = data['flux']
#variance_spectra_galaxy = data['var']
#sky_spectra_galaxy = data['sky']

In [ ]:
z = 0.405
lam_temp_ini = 3465.0       
lam_temp_fin = 9469.00
lam_range_min = lam_temp_ini*(1+z)
lam_range_max = lam_temp_fin*(1+z)
print(lam_range_min)
print(lam_range_max)

In [ ]:
lamda_galaxy_red = data_red['wav']
spectra_galaxy_red = data_red['flux']
variance_spectra_galaxy_red = data_red['var']
sky_spectra_galaxy_red = data_red['sky']
lamda_galaxy_range_red =  [np.min(lamda_galaxy_red), np.max(lamda_galaxy_red)]
#lamda_galaxy.tolist().index(4999.94676193628)
plt.plot(lamda_galaxy_red, spectra_galaxy_red, 'b')
#plt.figure()
plt.plot(lamda_galaxy_red, variance_spectra_galaxy_red*0.1, 'r')
#plt.figure()
#plt.plot(lamda_galaxy[:500], sky_spectra_galaxy[:500], 'g')

In [ ]:
# Calulating velocity scale

velocity_scale_red = vd.velocity_scale(lamda_galaxy_red)

# Calculating the parameter 'dv'

dv_red = vd.wav_dev(lamda_galaxy_red[0])

# Initial guess for velocity and velocity dispersion

c = 299792.458
vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
start = [vel, 200.0]  

# Logarithmically rebinning the galaxy spectra

median_spectra_gal_red = np.median(spectra_galaxy_red)       # median of the spectra_galaxy
spectra_galaxy_normalized_red = spectra_galaxy_red / median_spectra_gal_red
spectra_galaxy_rebinned_red, log_lamda_galaxy_red, v_red = util.log_rebin(lamda_galaxy_range_red, 
                                        spectra_galaxy_normalized_red, velscale=velocity_scale_red)
plt.plot(log_lamda_galaxy_red, spectra_galaxy_rebinned_red)
plt.title('logarithmically rebinned galaxy spectra')
plt.show()

## Noise

sigma_galaxy_spectra_red = np.sqrt(variance_spectra_galaxy_red) 
noise_red = sigma_galaxy_spectra_red / median_spectra_gal_red

#constant_noise = np.full_like(spectra_galaxy_rebinned, 0.02) 
noise_rebinned_red, log_lamda_noise_red, velo_scale_red = util.log_rebin(lamda_galaxy_range_red, noise_red,
                                                          velscale=velocity_scale_red)

plt.plot(log_lamda_noise_red, noise_rebinned_red)
plt.title('logarithmically rebinned noise')
#plt.plot(log_lamda_galaxy, spectra_galaxy_rebinned, label='logarithmically rebinned galaxy spectra')
#plt.plot(log_lamda_galaxy, constant_noise, label='constant noise')
#plt.legend()
plt.show()
#print(velo_scale)
#print(np.size(noise_new))


# Calculating the difference in sigma between the instrumental LSF of the galaxy spectra and templates.
# Here sigma of the instrument, 'sig_ins', has been obtained by performing Gaussian fit with several emission 
# lines from the galaxy spectra.

sigma_diff = vd.gen_sigma_diff(sig_ins=1.982, lam_gal=lamda_galaxy_red)


In [ ]:
v_red

In [ ]:
#%matplotlib inline
plt.plot(log_lamda_galaxy, spectra_galaxy_rebinned)

In [ ]:
sky_spectra_normalized_red = sky_spectra_galaxy_red / np.median(sky_spectra_galaxy_red)
sky_spectra_rebinned_red, log_lamda_sky_red, v_red = util.log_rebin(lamda_galaxy_range_red, 
                                                sky_spectra_normalized_red, velscale=velocity_scale_red)

In [ ]:
mask_region = [[(8.8351, 8.83976), (8.9344, 8.9508), (9.03385, 9.03853), (9.04731, 9.04993), (9.0919, 9.1035)],
              [(8.8351, 8.83976), (8.9344, 8.9808), (9.02787, 9.1035)],
              [(8.8351, 8.83976), (8.9344, 8.9808), (9.0127, 9.02541), (9.02787, 9.1035)],
              [(8.8282, 8.84105), (8.9344, 8.9808), (9.0131, 9.02541), (9.02787, 9.1035)], 
              [(8.7045, 8.7326), (8.8282, 8.84105), (8.9344, 8.9808), (9.0131, 9.02541), (9.02787, 9.1035)]] 
#(8.95506, 8.97243), (8.82944, ), (8.95506, 8.9808), (9.0839, 9.1035) (9.02787, 9.05235),

In [ ]:
templates_red = vd.gen_rebinned_templates(lib_path='../TEXT/*', temp_num=850, sigma_diff=sigma_diff,
                                      v=velocity_scale_red)

In [ ]:
for i, m in enumerate(mask_region):
    mask = vd.masking(m, log_lamda_galaxy)
    pp = ppxf(templates, spectra_galaxy_rebinned, noise_rebinned, velocity_scale, start, moments=4,
              plot=True, vsyst=dv, degree=4, mask=mask, lam=np.exp(log_lamda_galaxy), 
              sky=sky_spectra_rebinned)
    plt.figure()

In [ ]:
degree = np.arange(4,21)
#velocity_dispersion = np.zeros(len(degree))
velocity_dispersion = []
error_corrected = [] #np.zeros(len(degree))
#error = np.zeros(len(degree))
error = []
#mask = vd.masking(mask_region[0], log_lamda_galaxy)
for j, m in enumerate(mask_region):
    mask = vd.masking(m, log_lamda_galaxy)
    vel_dis = np.zeros(len(degree))
    err = np.zeros(len(degree))
    err_corr = np.zeros(len(degree))
    for i in range(len(degree)):
        pp = ppxf(templates, spectra_galaxy_rebinned, noise_rebinned, velocity_scale, start, moments=4,
                 plot=False, degree=degree[i], vsyst=dv, mask= mask, lam=np.exp(log_lamda_galaxy))#,
                 #sky=sky_spectra_rebinned)
        vel_dis[i] = pp.sol[1]
        err[i] = pp.error[1]
        err_corr[i] = pp.error[1]*np.sqrt(pp.chi2)
    velocity_dispersion.append(vel_dis)
    error.append(err)
    error_corrected.append(err_corr)
#plt.figure()
#print('degree : %d' %degree[i])

In [ ]:
color = ['b', 'g', 'r', 'k', 'y']
label = ['mask_reg_1', 'mask_reg_2', 'mask_reg_3', 'mask_reg_4', 'mask_reg_5']
for i, p in enumerate(velocity_dispersion):
    plt.plot(degree, p, '.-', ms=10, color=color[i], label=label[i])
    
#plt.plot(degree, velocity_dispersion, '.')
plt.xlabel('degree')
plt.ylabel('velocity dispersion')
plt.legend()
#plt.title('with mask region 2')
#plt.ylim(280, 400)
#plt.figure()

In [ ]:
color = ['b', 'g', 'r', 'k', 'y']
label = ['mask_reg_1', 'mask_reg_2', 'mask_reg_3', 'mask_reg_4', 'mask_reg_5']
for i, p in enumerate(error):
    plt.plot(degree, p, '.-', color=color[i], label=label[i])
    
#plt.plot(degree, velocity_dispersion, '.')
plt.xlabel('degree')
plt.ylabel('error')
plt.legend()
#plt.title('with mask region 2')
plt.ylim(5, 12)
#plt.figure()

In [ ]:
degree = np.arange(4,18)
velocity_dispersion = np.zeros(len(degree))
error_corrected = np.zeros(len(degree))
error = np.zeros(len(degree))
mask = vd.masking(mask_region[0], log_lamda_galaxy)
for i in range(len(degree)):
    pp = ppxf(templates, spectra_galaxy_rebinned, noise_rebinned, velocity_scale, start, moments=4,
         plot=True, degree=degree[i], vsyst=dv, mask= mask, lam=np.exp(log_lamda_galaxy))
    velocity_dispersion[i] = pp.sol[1]
    error[i] = pp.error[1]
    error_corrected[i] = pp.error[1]*np.sqrt(pp.chi2)
    plt.figure()
    print('degree : %d' %degree[i])

In [ ]:
plt.plot(degree, velocity_dispersion, '.')
plt.xlabel('degree')
plt.ylabel('velocity dispersion')
#plt.title('with mask region 2')
#plt.ylim(280, 400)
plt.figure()

In [ ]:
plt.plot(degree, error, '.')
plt.xlabel('degree')
plt.ylabel('error')
#plt.title('with mask region 2')
#plt.ylim(7, 11)
plt.figure()

In [ ]:
degree = np.arange(4,21)
velocity_dispersion = np.zeros(len(degree))
error_corrected = np.zeros(len(degree))
error = np.zeros(len(degree))
mask = vd.masking(mask_region[0], log_lamda_galaxy)
for i in range(len(degree)):
    pp = ppxf(templates, spectra_galaxy_rebinned, noise_rebinned, velocity_scale, start, moments=4,
         plot=True, degree=degree[i], vsyst=dv, mask= mask, lam=np.exp(log_lamda_galaxy), 
         sky=sky_spectra_rebinned)
    velocity_dispersion[i] = pp.sol[1]
    error[i] = pp.error[1]
    error_corrected[i] = pp.error[1]*np.sqrt(pp.chi2)
    plt.figure()
    print('degree : %d' %degree[i])

In [ ]:
plt.plot(degree, velocity_dispersion, '.')
plt.xlabel('degree')
plt.ylabel('velocity dispersion')
#plt.title('with mask region 2')
plt.ylim(280, 410)
plt.figure()

In [ ]:
plt.plot(degree, error, '.')
plt.xlabel('degree')
plt.ylabel('error')
#plt.title('with mask region 2')
plt.ylim(8, 11)
plt.figure()

In [ ]:
coadded_spectra_blue = deimosmask1d.DeimosMask1d('../galaxy_spectra/0712m4-blue-coadd_latest.fits')

In [ ]:
specid_blue = [] #np.zeros(coadded_spectra.nspec)
info = coadded_spectra_blue.slitinfo
#print(info)
for i in range(coadded_spectra_blue.nspec):
    spec_id = '%d_%d_%d_%d' % (info['det'][i], info['slitid'][i], info['objid'][i], info['spatloc'][i])
    specid_blue.append(spec_id)
    print(spec_id)

In [ ]:
gal_spec_id_blue = specid_blue[10]
print('galaxy_spectra_id: %s' % (gal_spec_id_blue))
data_blue =  coadded_spectra_blue[gal_spec_id_blue]
#print(data)
data_blue.smooth(7)
data_blue.mark_lines('abs', z=0.405, usesmooth=True)
#lamda_galaxy = data['wav']
#spectra_galaxy = data['flux']
#variance_spectra_galaxy = data['var']
#sky_spectra_galaxy = data['sky']

In [ ]:
data_blue.save('0712_lens_spectra_blue_side.txt', outformat='text')

In [ ]:
d = spec1d.Spec1d('0712_lens_spectra_blue_side.txt')
d.smooth(7)
d.mark_lines('abs', z=0.405, usesmooth=True)

In [ ]:
data_blue.find_dispave(data_blue['wav'])

In [ ]:
data_red.find_dispave(data_red['wav'])

In [ ]:
len(data_red['wav'])

In [ ]:
a = np.round(np.linspace(data_blue['wav'][0], data_blue['wav'][-1], num=3199), 2)
print(a)
data_blue.find_dispave(a)

In [ ]:
data_blue.resample(owave=a)
data_blue.save('resampled_lens_galaxy_blue_side.txt', outformat='text', useresamp=True)

In [ ]:
d